## Setup

In [1]:
!pip install tensorflow-gpu==2.0 tensorflow_datasets gpustat -Uq

**About**

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/6/6d/Nvidia_image_logo.svg/200px-Nvidia_image_logo.svg.png" width="90px" align="right" style="margin-right: 0px;">

This notebook is put together by Timothy Liu (`timothyl@nvidia.com`) for the [**PyCon SG**](https://pycon.sg/) 2019 tutorial on [**Improving Deep Learning Performance in TensorFlow**](https://github.com/NVAITC/pycon-sg19-tensorflow-tutorial).

**Acknowledgements**

* This notebook uses some materials adapted from TensorFlow documentation.
* This notebook uses the [Oxford IIT Pet Dataset](http://www.robots.ox.ac.uk/~vgg/data/pets/) ([TensorFlow Datasets page](https://www.tensorflow.org/datasets/catalog/oxford_iiit_pet)).

**Dataset Citation**

```
@InProceedings{parkhi12a,
  author       = "Parkhi, O. M. and Vedaldi, A. and Zisserman, A. and Jawahar, C.~V.",
  title        = "Cats and Dogs",
  booktitle    = "IEEE Conference on Computer Vision and Pattern Recognition",
  year         = "2012",
}
```

In [2]:
import multiprocessing

import tensorflow
print("TensorFlow version:", tensorflow.__version__)

import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

TensorFlow version: 2.0.0


In [3]:
import time

class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []
    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()
    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

# Pets Classification with TensorFlow

In [4]:
!gpustat

jupyter-admin        Fri Oct 11 16:48:49 2019  410.104
[0] Tesla T4         | 43'C,   0 % |     0 / 15079 MB |


In [5]:
import tensorflow.keras.layers as layers
from tensorflow.keras.applications.resnet50 import ResNet50

def create_model(img_size=(224,224), num_class=2, train_base=True):
    input_layer = layers.Input(shape=(img_size[0],img_size[1],3))
    base = ResNet50(input_tensor=input_layer,
                    include_top=False,
                    weights="imagenet")
    base.trainable = train_base
    x = base.output
    x = layers.GlobalAveragePooling2D()(x)
    preds = layers.Dense(num_class, activation="softmax")(x)
    return tf.keras.models.Model(inputs=input_layer, outputs=preds)

In [6]:
(train_dataset, test_dataset), info = tfds.load(name="oxford_iiit_pet:3.*.*",
                                                split=["train", "test"],
                                                shuffle_files=True,
                                                as_supervised=True,
                                                with_info=True)

num_class = info.features["label"].num_classes
num_train = info.splits["train"].num_examples
num_test  = info.splits["test"].num_examples

Shuffling and writing examples to /home/jovyan/tensorflow_datasets/oxford_iiit_pet/3.0.0.incompleteN3JX06/oxford_iiit_pet-train.tfrecord


Shuffling and writing examples to /home/jovyan/tensorflow_datasets/oxford_iiit_pet/3.0.0.incompleteN3JX06/oxford_iiit_pet-test.tfrecord



Dataset oxford_iiit_pet downloaded and prepared to /home/jovyan/tensorflow_datasets/oxford_iiit_pet/3.0.0. Subsequent calls will reuse this data.


In [7]:
IMG_SIZE = (224, 224)

@tf.function
def format_train_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, IMG_SIZE)
    # perform image augmentation with tf.image
    image = tf.image.flip_left_right(image)
    image = tf.image.random_brightness(image, 0.1)
    return image, tf.one_hot(label, num_class)

@tf.function
def format_eval_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, IMG_SIZE)
    return image, tf.one_hot(label, num_class)

In [8]:
BATCH_SIZE = 80
N_THREADS = multiprocessing.cpu_count()
PREFETCH_COUNT = 8

train_dataset = train_dataset.shuffle(1024)
train_dataset = train_dataset.repeat(-1)
train_dataset = train_dataset.map(format_train_example,
                                  num_parallel_calls=N_THREADS)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(PREFETCH_COUNT)

In [9]:
test_dataset = test_dataset.map(format_eval_example,
                                num_parallel_calls=N_THREADS)
test_dataset = test_dataset.repeat(-1)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [10]:
model = create_model(IMG_SIZE, num_class, train_base=True)
opt = tf.keras.optimizers.Adam()

model.compile(loss="categorical_crossentropy",
              optimizer=opt,
              metrics=["acc"])

#model.summary()

94773248/94765736 [==============================] - 6s 0us/step


In [11]:
steps_per_epoch = num_train//BATCH_SIZE
steps_test = num_test//BATCH_SIZE

time_callback = TimeHistory()

In [12]:
model.fit(train_dataset, steps_per_epoch=steps_per_epoch,
          epochs=5, callbacks=[time_callback], verbose=1)

Train for 46 steps
Epoch 1/5
46/46 [==============================] - 63s 1s/step - loss: 1.8270 - acc: 0.4777
Epoch 2/5
46/46 [==============================] - 36s 773ms/step - loss: 0.6667 - acc: 0.7905
Epoch 3/5
46/46 [==============================] - 38s 832ms/step - loss: 0.3448 - acc: 0.8889
Epoch 4/5
46/46 [==============================] - 37s 807ms/step - loss: 0.2233 - acc: 0.9321
Epoch 5/5
46/46 [==============================] - 37s 812ms/step - loss: 0.1526 - acc: 0.9552


In [13]:
# There is currently a bug with model.evaluate()
# Follow: https://github.com/tensorflow/tensorflow/issues/33090

#model.evaluate(train_dataset, steps=steps_per_epoch)

In [14]:
epoch_time = min(time_callback.times)
img_per_sec = num_train//epoch_time

print("Peak Img/s:", img_per_sec)

Peak Img/s: 103.0
